# Machine Learning
-----


In this notebook, we'll discuss how to formulate a research question in the machine learning framework; how to build, evaluate, compare, and select models; and how to reasonably and accurately interpret model results. You'll also get hands-on experience using the `scikit-learn` package in Python.

This tutorial is based on chapter 6 of [Big Data and Social Science](https://github.com/BigDataSocialScience/).

## Glossary of Terms


There are a number of terms specific to Machine Learning that you will find repeatedly in this notebook. 

- **Learning**: In Machine Learning, you'll hear about "learning a model." This is what you probably know as 
*fitting* or *estimating* a function, or *training* or *building* a model. These terms are all synonyms and are 
used interchangeably in the machine learning literature.
- **Examples**: These are what you probably know as *data points* or *observations* or *rows*. 
- **Features**: These are what you probably know as *independent variables*, *attributes*, *predictors*, 
or *explanatory variables.*
- **Underfitting**: This happens when a model is too simple and does not capture the structure of the data well 
enough.
- **Overfitting**: This happens when a model is too complex or too sensitive to the noise in the data; this can
result in poor generalization performance, or applicability of the model to new data. 
- **Regularization**: This is a general method to avoid overfitting by applying additional constraints to the model. 
For example, you can limit the number of features present in the final model, or the weight coefficients applied
to the (standardized) features are small.
- **Supervised learning** involves problems with one target or outcome variable (continuous or discrete) that we want
to predict, or classify data into. Classification, prediction, and regression fall into this category. We call the
set of explanatory variables $X$ **features**, and the outcome variable of interest $Y$ the **label**.
- **Unsupervised learning** involves problems that do not have a specific outcome variable of interest, but rather
we are looking to understand "natural" patterns or groupings in the data - looking to uncover some structure that 
we do not know about a priori. Clustering is the most common example of unsupervised learning, another example is 
principal components analysis (PCA).



## Setup
We'll be using [`scikit-learn`](http://scikit-learn.org) for the machine learning models.

In [ ]:
%pylab inline
import os
import glob
import pandas as pd
import time
import sklearn
import seaborn as sns
from sklearn.metrics import precision_recall_curve,roc_curve, auc
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.ensemble import (RandomForestClassifier, ExtraTreesClassifier,
                              GradientBoostingClassifier,
                              AdaBoostClassifier)
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_curve, roc_auc_score, auc
from sklearn import preprocessing

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

## The Machine Learning Process

The Machine Learning Process is as follows:

- **Understand the problem and goal.** *This sounds obvious but is often nontrivial.* Problems typically start as vague descriptions of a goal - improving health outcomes, increasing graduation rates, understanding the effect of a variable *X* on an outcome *Y*, etc. It is really important to work with people who understand the domain being studied to dig deeper and define the problem more concretely. What is the analytical formulation of the metric that you are trying to optimize?
- **Formulate it as a machine learning problem.** Is it a classification problem or a regression problem? Is the goal to build a model that generates a ranked list prioritized by risk, or is it to detect anomalies as new data come in? Knowing what kinds of tasks machine learning can solve will allow you to map the problem you are working on to one or more machine learning settings and give you access to a suite of methods.
- **Data exploration and preparation.** Next, you need to carefully explore the data you have. What additional data do you need or have access to? What variable will you use to match records for integrating different data sources?  What variables exist in the data set? Are they continuous or categorical? What about missing values? Can you use the variables in their original form, or do you need to alter them in some way?
- **Feature engineering.** In machine learning language, what you might know as independent variables or predictors or factors or covariates are called "features." Creating good features is probably the most important step in the machine learning process. This involves doing transformations, creating interaction terms, or aggregating over data points or over time and space.
- **Method selection.** Having formulated the problem and created your features, you now have a suite of methods to choose from. It would be great if there were a single method that always worked best for a specific type of problem. Typically, in machine learning, you take a variety of methods and try them, empirically validating which one is the best approach to your problem.
- **Evaluation.** As you build a large number of possible models, you need a way choose the best among them. We'll cover methodology to validate models on historical data and discuss a variety of evaluation metrics. 
- **Deployment.** Once you have selected the best model and validated it using historical data as well as a field trial, you are ready to put the model into practice. You still have to keep in mind that new data will be coming in, and the model might change over time.

You're probably used to fitting models in social science classes. In those cases, you probably had a hypothesis or theory about the underlying process that gave rise to your data, chose an appropriate model based on prior knowledge and fit it using least squares, and used the resulting parameter or coefficient estimates (or confidence intervals) for inference. This type of modeling is very useful for *interpretation*.

In machine learning, our primary concern is *generalization*. This means that:
- **We care less about the structure of the model and more about the performance** This means that we'll try out a whole bunch of models at a time and choose the one that works best, rather than determining which model to use ahead of time. We can then choose to select a *suboptimal* model if we care about a specific model type. 
- **We don't (necessarily) want the model that best fits the data we've *already seen*,** but rather the model that will perform the best on *new data*. This means that we won't gauge our model's performance using the same data that we used to fit the model (e.g., sum of squared errors or $R^2$), and that "best fit" or accuracy will most often *not* determine the best model.  
- **We can include a lot of variables in to the model.** This may sound like the complete opposite of what you've heard in the past, and it can be hard to swallow. 

## Problem Formulation

First, turning something into a real objective function. What do you care about? Do you have data on that thing? What action can you take based on your findings? Do you risk introducing any bias based on the way you model something? 

### Four Main Types of ML Tasks for Policy Problems

- **Description**: [How can we identify and respond to the most urgent online government petitions?](https://dssg.uchicago.edu/project/improving-government-response-to-citizen-requests-online/)
- **Prediction**: [Which students will struggle academically by third grade?](https://dssg.uchicago.edu/project/predicting-students-that-will-struggle-academically-by-third-grade/)
- **Detection**: [Which police officers are likely to have an adverse interaction with the public?](https://dssg.uchicago.edu/project/expanding-our-early-intervention-system-for-adverse-police-interactions/)
- **Behavior Change**: [How can we prevent juveniles from interacting with the criminal justice system?](https://dssg.uchicago.edu/project/preventing-juvenile-interactions-with-the-criminal-justice-system/)
  
### Our Machine Learning Problems
> We will first use an unsupervised method to cluster grnats and then see if we can predict high impact/breakthrough grants based on the information we have in the project data using a supervised model

This is an example of a *binary prediction classification problem*. In this case, we are trying to predict a binary outcome: whether a patent is a impactful patent or not.

Note that the way the outcome is defined is somewhat arbitrary. You may decide to define the label in a different way, or consider different features.

## Data Exploration and Preparation

During the first classes, we have explored the data, linked different data sources, and created new variables. The first steps of creating a machine learning model are exactly that. 

1. **Creating labels**: Labels are the dependent variables, or *Y* variables, that we are trying to predict. In the machine learning framework, labels are often *binary*: true or false, encoded as 1 or 0. This outcome variable is named `label`.

1. **Decide on feature**: Our features are our independent variables or predictors. Good features make machine learning systems effective. The better the features the easier it is the capture the structure of the data. You generate features using domain knowledge. In general, it is better to have more complex features and a simpler model rather than vice versa. Keeping the model simple makes it faster to train and easier to understand rather then extensively searching for the "right" model and "right" set of parameters. Machine Learning Algorithms learn a solution to a problem from sample data. The set of features is the best representation of the sample data to learn a solution to a problem.

1. **Feature engineering** is the process of transforming raw data into features that better represent the underlying problem/data/structure  to the predictive models, resulting in improved model accuracy on unseen data." ( from [Discover Feature Engineering](http://machinelearningmastery.com/discover-feature-engineering-how-to-engineer-features-and-how-to-get-good-at-it/) ).  In text, for example, this might involve deriving traits of the text like word counts, verb counts, or topics to feed into a model rather than simply giving it the raw text. Example of feature engineering are: 
    - **Transformations**, such a log, square, and square root.
    - **Dummy (binary) variables**, sometimes known as *indicator variables*, often done by taking categorical variables (such as industry) which do not have a numeric value, and adding them to models as a binary value.
    - **Discretization**. Several methods require features to be discrete instead of continuous. This is often done by binning, which you can do by equal width, deciles, Fisher-Jenks, etc. 
    - **Aggregation.** Aggregate features often constitute the majority of features for a given problem. These use different aggregation functions (*count, min, max, average, standard deviation, etc.*) which summarize several values into one feature, aggregating over varying windows of time and space. For example, we may want to calculate the *number* (and *min, max, mean, variance*, etc.) of crimes within an *m*-mile radius of an address in the past *t* months for varying values of *m* and *t*, and then use all of them as features.

1. **Cleaning data**: To run the `scikit-learn` set of models your input dataset must have no missing variables.

1. **Imputing values to missing or irrelevant data**: Once the features are created, always check to make sure the values make sense. You might have some missing values, or impossible values for a given variable (negative values, major outliers). If you have missing values you should think hard about what makes the most sense for your problem; you may want to replace with `0`, the median or mean of your data, or some other value.

1. **Scaling features**: Certain models will have an issue with features on different scales. For example, an individual's age is typically a number between 0 and 100 while earnings can be number between 0 and 1000000 (or higher). In order to circumvent this problem, we can scale our features to the same range (eg [0,1]).

# Load Data 

In [ ]:
# My data directory
data_dir = "/home/jovyan/Yandex.Disk/BigDataPubPol/data"
print( "The data directory for the class data is " + data_dir )
# Now we are switching into the folder that has the patent data
os.chdir(data_dir + "/projects")

In [ ]:
# Generate an empty dataframe that will hold all the patent data we have
grants_1012 = pd.DataFrame([])

# Now loop through each file in the folder that starts with patent
# And append it to the dataframe that we created above

for i in range(2010,2013):
    g = pd.read_csv("FedRePORTER_PRJ_C_FY{}.csv".format(i), low_memory=False, skipinitialspace=True,
                    usecols=['PROJECT_ID','PROJECT_TERMS','PROJECT_TITLE','DEPARTMENT','AGENCY',
                             'PROJECT_START_DATE','PROJECT_END_DATE','CONTACT_PI_PROJECT_LEADER', 
                             'CONGRESSIONAL_DISTRICT','ORGANIZATION_NAME','ORGANIZATION_CITY',
                             'ORGANIZATION_STATE','ORGANIZATION_ZIP','ORGANIZATION_COUNTRY',
                             'FY','FY_TOTAL_COST'])
    grants_1012 = grants_1012.append(g)

In [ ]:
# List the colums and frequency counts
grants_1012.count()

In [ ]:
# How does our data look like?
grants_1012.head()

# Problem Formulation - Unsupervised Learning

Unsupervised Learning is a class of Machine Learning techniques to find the patterns in data. The data given to unsupervised algorithm are not labelled, which means only the input variables(X) are given with no corresponding output variables. In unsupervised learning, the algorithms are left to themselves to discover interesting structures in the data. So a problem formulation in our case would be identifying similar research projects. 

# Building a Model

Let's run a K-Means cluster analyses.

In [ ]:
# What are our features again?
print(grants_1012.columns.values)

In [ ]:
# K-Means does not support missing vlaues. You need clean data (delete or impute)
print(grants_1012.isnull().sum())

In [ ]:
# You have to carefully think how to impute missing values or if you want to drop them
# Beacause it takes too long to clean data we are dropping them for now
grants_1012 = grants_1012.dropna()

In [ ]:
# It is important to understand the features (categorical vs. numerical)
grants_1012.info()

We need two lines of code. In the first line, we create a KMeans object and pass it 3 as value for n_clusters parameter (you can set this parameter as you wish). Next, we call the fit method on kmeans and pass the data we want to cluster, which in this case is the grants data.

For a cluster analysis (k-means we need numerical data).But we can create some features.

In [ ]:
# For terms, title we can count the words
# For terms, title we can count the words
grants_1012['tit_len'] = grants_1012['PROJECT_TITLE'].str.split().str.len()
grants_1012['terms_num'] = grants_1012['PROJECT_TERMS'].str.split(';').str.len()

# We just need the yearfo start and end
grants_1012['project_start'] = grants_1012['PROJECT_START_DATE'].str.extract('(\d\d\d\d)', expand=True).astype(int)
grants_1012['project_end'] = grants_1012['PROJECT_END_DATE'].str.extract('(\d\d\d\d)', expand=True).astype(int)

In [ ]:
# Keep only the numerical variables
df_cluster = grants_1012[['project_start','project_end','FY_TOTAL_COST','FY','tit_len','terms_num']]
df_cluster

In [ ]:
# Run cluster algorithm
kmeans = KMeans(n_clusters=3) 
kmeans.fit(df_cluster)
labels = kmeans.predict(df_cluster)

In [ ]:
# Check centroid values the algorithm generated for the final clusters
print(kmeans.cluster_centers_) 

In [ ]:
# Add back to original data
df_cluster['clusters'] = labels

In [ ]:
# Lets analyze the clusters
print(df_cluster.groupby(['clusters']).mean())

This tells us what the differences between the clusters are. It shows mean values of the attribute per each cluster.

In [ ]:
#Scatter plot of Number of Terms and Total Cost
sns.lmplot('terms_num', 'FY_TOTAL_COST', 
           data=df_cluster, 
           fit_reg=False, 
           hue="clusters",  
           scatter_kws={"marker": "D", 
                        "s": 100})
plt.title('Terms vs Funding Amount')
plt.xlabel('Number of Terms')
plt.ylabel('Funding')

# Quality Checks

You can check the quality of your cluster by invetigating how well it fits to the data. A good cluster has all the data close together and not spread out. We can measure the distance from the centroid for each cluster. Lower is better.

In [ ]:
print(kmeans.inertia_)

For each k value, we will initialise k-means and use the inertia attribute to identify the sum of squared distances of samples to the nearest cluster centre. As k increases, the sum of squared distance tends to zero. Imagine we set k to its maximum value n (where n is number of samples) each sample will form its own cluster meaning sum of squared distances equals zero.

In [ ]:
df_cluster.drop(['clusters'], axis=1, inplace=True)

In [ ]:
Sum_of_squared_distances = []
K = range(1,15)
for k in K:
    km = KMeans(n_clusters=k)
    km = km.fit(df_cluster)
    Sum_of_squared_distances.append(km.inertia_)

In [ ]:
# Plot the result: Elbow plot
plt.plot(K, Sum_of_squared_distances, 'bx-')
plt.xlabel('k')
plt.ylabel('Sum_of_squared_distances')
plt.title('Elbow Method For Optimal k')
plt.show()

# Problem Formulation - Supervised Learning
Now lets use a supervised model to seeif we can predict grants with high impact.  We first need to define our outcome. What is a grant. with high impact? There are many ways to measure this. For now we will go with the funding amount and say that whatever is in the tp 75-perecentile has high impact. Then we have to build the features and build our test and training data.

In [ ]:
grants_1012.head()

# Building a Model

We need to devide our dataset into **features** (predictors, or independent variables, or $X$ variables) and **labels** (dependent variables, or $Y$ variables).  

# Creating Labels

Labels are the dependent variables, or *Y* variables, that we are trying to predict. In the machine learning framework, your labels are usually *binary*: true or false, encoded as 1 or 0. In this case, our label is whether a grant has published a paper. So our label *Y* is 0 if there are no publications associated with a grant, and is 1 if a grant was able to publish a paper.

In [ ]:
grants_1012['FY_TOTAL_COST'].describe()

In [ ]:
grants_1012['Y'] = np.where(grants_1012['FY_TOTAL_COST'] >= 4.144582e+05 , 1, 0)
grants_1012['Y'].value_counts()

# Feature Generation

Our features are our independent variables or predictors. Good features make machine learning systems effective. 
The better the features the easier it is the capture the structure of the data. You generate features using domain knowledge. In general, it is better to have more complex features and a simpler model rather than vice versa. Keeping the model simple makes it faster to train and easier to understand rather then extensively searching for the "right" model and "right" set of parameters. 

Our features are the following

- `state`: The state in which the applicant organization is located (we need to make dummies)

- `project_start`: Start of project, dummies

- `project_end`: End of project, dummies

- `terms_num`: Number of words in project terms

- `tit_len`: Length of Project Title

- `fy`: Year, we need to make a dummy here too

- `agency`: The funding agency, we need to make a dummy here too

In [ ]:
df = grants_1012[['project_start','project_end','Y','FY','tit_len','terms_num','ORGANIZATION_STATE','AGENCY']]

In [ ]:
# We need to creat dummy variables for state and organization name
df = pd.get_dummies(df, columns=['ORGANIZATION_STATE', 'FY', 'AGENCY', 'project_start', 'project_end'])

In [ ]:
df.count()

## Model Fitting

It's not enough to just build the model; we're going to need a way to know whether or not it's working. Convincing others of the quality of results is often the *most challenging* part of an analysis.  Making repeatable, well-documented work with clear success metrics makes all the difference.

To convince ourselves - and others - that our modeling results will generalize, we need to hold
some data back (not using it to train the model), then apply our model to that hold-out set and "blindly" predict, comparing the model's predictions to what we actually observed. This is called **cross-validation**, and it's the best way we have to estimate how a model will perform on *entirely* novel data. We call the data used to build the model the **training set**, and the rest the **test set**.

For our project we will start dividing the sample to have a training and test set. There are mulriple ways to define training and test set. You can try different definitions on your own. 

In [ ]:
# We can easily split data by using a pre-defined function from the sklearn package
df_train, df_test = train_test_split(df, test_size=0.8)

In [ ]:
df_train.describe()

In [ ]:
df_test.describe()

### Imputation 

It is important to to do a quick check of our matrix to see if we have any outlier values. We might have some problems with the publication count. However in our example it is fine because we are only looking at binary outcomes. Also you want to investigate missing values. You can't have missing values sickit models.

### Class Balancing

Let's check how much data we still have and how many end in publications in our training dataset. We don't necessarily need to have a perfect 50-50 balance, but it's good to know what the "baseline" is in our dataset, to be able to intelligently evaluate our performance. If you look at our Y you can see that not that many grant result in publications.

In [ ]:
print('Number of rows: {}'.format(df_train.shape[0]))
df_train['Y'].value_counts(normalize=True)

Let's take a look at our testing set. 

In [ ]:
print('Number of rows: {}'.format(df_test.shape[0]))
df_test['Y'].value_counts(normalize=True)

### Investigate our data

We can use visualizations to find trends and patterns in our data. 

In [ ]:
ax = sns.boxplot(x="Y", y="terms_num", data=df_train)

In [ ]:
ax = sns.boxplot(x="Y", y="tit_len", data=df_train)

### Split into features and labels

In [ ]:
org = [col for col in df_train if col.startswith('ORGANIZATION')]
project = [col for col in df_train if col.startswith('project')]

sel_features = ['FY_2010','FY_2011','FY_2012', 'terms_num','tit_len', 'AGENCY_AHRQ','AGENCY_ALLCDC','AGENCY_ARS','AGENCY_CDMRP','AGENCY_CNRM','AGENCY_FDA','AGENCY_FS',                
'AGENCY_IES','AGENCY_NASA','AGENCY_NIDILRR','AGENCY_NIFA','AGENCY_NIH','AGENCY_NSF'] + org + project
sel_label = 'Y'

In [ ]:
# use conventions typically used in python scikitlearn

X_train = df_train[sel_features].values
y_train = df_train[sel_label].values
X_test = df_test[sel_features].values
y_test = df_test[sel_label].values

# Model Selection

## Model Evaluation 

In this phase, you take the predictors from your test set and apply your model to them, then assess the quality of the model by comparing the *predicted values* to the *actual values* for each record in your testing data set. 

- **Performance Estimation**: How well will our model do once it is deployed and applied to new data?

Now let's use the model we just fit to make predictions on our test dataset, and see what our accuracy score is:

Python's [`scikit-learn`](http://scikit-learn.org/stable/) is a commonly used, well documented Python library for machine learning. This library can help you split your data into training and test sets, fit models and use them to predict results on new data, and evaluate your results.

We will start with the simplest [`LogisticRegression`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) model and see how well that does.

You can use any number of metrics to judge your models, but we'll use [`accuracy_score()`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html) (ratio of correct predictions to total number of predictions) as our measure.

In [ ]:
# Let's fit a model
from sklearn import linear_model
model = linear_model.LogisticRegression(penalty='l1', C=1e5)
model.fit( X_train, y_train )
print(model)
#https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

When we print the model results, we see different parameters we can adjust as we refine the model based on running it against test data (values such as `intercept_scaling`, `max_iters`, `penalty`, and `solver`).  

Penalty L1: Lasso shrinks the less important feature’s coefficient to zero thus, removing some feature altogether. So, this works well for feature selection in case we have a huge number of features.

C: Inverse of regularization strength; must be a positive float. Smaller values specify stronger regularization.

To adjust these parameters, one would alter the call that creates the `LogisticRegression()` model instance, passing it one or more of these parameters with a value other than the default.  So, to re-fit the model with `max_iter` of 1000, `intercept_scaling` of 2, and `solver` of "lbfgs" (pulled from thin air as an example), you'd create your model as follows:

    model = LogisticRegression( max_iter = 1000, intercept_scaling = 2, solver = "lbfgs" )

The basic way to choose values for, or "tune," these parameters is the same as the way you choose a model: fit the model to your training data with a variety of parameters, and see which perform the best on the test set. An obvious drawback is that you can also *overfit* to your test set; in this case, you can alter your method of cross-validation.



# Model Evaluation 

Machine learning models usually do not produce a prediction (0 or 1) directly. Rather, models produce a score between 0 and 1 (that can sometimes be interpreted as a probability), which lets you more finely rank all of the examples from *most likely* to *least likely* to have label 1 (positive). This score is then turned into a 0 or 1 based on a user-specified threshold. For example, you might label all examples that have a score greater than 0.5 (1/2) as positive (1), but there's no reason that has to be the cutoff. 

In [ ]:
#  from our "predictors" using the model.
y_scores = model.predict_proba(X_test)[:,1]

In [ ]:
y_scores

Let's take a look at the distribution of scores and see if it makes sense to us. 

In [ ]:
sns.distplot(y_scores, kde=False, rug=False)

In [ ]:
df_test['y_score'] = y_scores

In [ ]:
df_test[['y_score']].head()

Tools like `sklearn` often have a default threshold of 0.5, but a good threshold is selected based on the data, model and the specific problem you are solving. As a trial run, let's set a threshold of 0.5. 

In [ ]:
calc_threshold = lambda x,y: 0 if x < y else 1 
predicted = np.array( [calc_threshold(score,0.50) for score in y_scores] )
expected = y_test

In [ ]:
sns.distplot(expected, kde=False, rug=False)

## Confusion Matrix

Once we have tuned our scores to 0 or 1 for classification, we create a *confusion matrix*, which  has four cells: true negatives, true positives, false negatives, and false positives. Each data point belongs in one of these cells, because it has both a ground truth and a predicted label. If an example was predicted to be negative and is negative, it's a true negative. If an example was predicted to be positive and is positive, it's a true positive. If an example was predicted to be negative and is positive, it's a false negative. If an example was predicted to be positive and is negative, it's a false negative.

In [ ]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(expected,predicted)
print(conf_matrix)

The count of true negatives is `conf_matrix[0,0]`, false negatives `conf_matrix[1,0]`, true positives `conf_matrix[1,1]`, and false_positives `conf_matrix[0,1]`.

Accuracy is the ratio of the correct predictions (both positive and negative) to all predictions. 
$$ Accuracy = \frac{TP+TN}{TP+TN+FP+FN} $$

In [ ]:
# generate an accuracy score by comparing expected to predicted.
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(expected, predicted)
print( "Accuracy = " + str( accuracy ) )

Two additional metrics that are often used are **precision** and **recall**. 

Precision measures the accuracy of the classifier when it predicts an example to be positive. It is the ratio of correctly predicted positive examples to examples predicted to be positive. 

$$ Precision = \frac{TP}{TP+FP}$$

Recall measures the accuracy of the classifier to find positive examples in the data. 

$$ Recall = \frac{TP}{TP+FN} $$

By selecting different thresholds we can vary and tune the precision and recall of a given classifier. A conservative classifier (threshold 0.99) will classify a case as 1 only when it is *very sure*, leading to high precision. On the other end of the spectrum, a low threshold (e.g. 0.01) will lead to higher recall. 

In [ ]:
from sklearn.metrics import precision_score, recall_score
precision = precision_score(expected, predicted)
recall = recall_score(expected, predicted)
print( "Precision = " + str( precision ) )
print( "Recall= " + str(recall))

If we care about our whole precision-recall space, we can optimize for a metric known as the **area under the curve (AUC-PR)**, which is the area under the precision-recall curve. The maximum AUC-PR is 1. 

In [ ]:
def plot_precision_recall(y_true,y_score):
    """
    Plot a precision recall curve
    
    Parameters
    ----------
    y_true: ls
        ground truth labels
    y_score: ls
        score output from model
    """
    precision_curve, recall_curve, pr_thresholds = precision_recall_curve(y_true,y_score)
    plt.plot(recall_curve, precision_curve)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    auc_val = auc(recall_curve,precision_curve)
    print('AUC-PR: {0:1f}'.format(auc_val))
    plt.show()
    plt.clf()

In [ ]:
plot_precision_recall(expected, y_scores)

## Precision and Recall at k%

If we only care about a specific part of the precision-recall curve we can focus on more fine-grained metrics. For instance, say there is a year where an agency has funding constraints. They have less money to allocate in this given year. In that case, we would want to prioritize the 1% of applicants who are *most likely* to publish, and it doesn't matter too much how accurate we are on the ones who aren't very likely to publish. We can then focus on optimizing our **precision at 1%**.

In [ ]:
def plot_precision_recall_n(y_true, y_prob, model_name):
    """
    y_true: ls 
        ls of ground truth labels
    y_prob: ls
        ls of predic proba from model
    model_name: str
        str of model name (e.g, LR_123)
    """
    from sklearn.metrics import precision_recall_curve
    y_score = y_prob
    precision_curve, recall_curve, pr_thresholds = precision_recall_curve(y_true, y_score)
    precision_curve = precision_curve[:-1]
    recall_curve = recall_curve[:-1]
    pct_above_per_thresh = []
    number_scored = len(y_score)
    for value in pr_thresholds:
        num_above_thresh = len(y_score[y_score>=value])
        pct_above_thresh = num_above_thresh / float(number_scored)
        pct_above_per_thresh.append(pct_above_thresh)
    pct_above_per_thresh = np.array(pct_above_per_thresh)
    plt.clf()
    fig, ax1 = plt.subplots()
    ax1.plot(pct_above_per_thresh, precision_curve, 'b')
    ax1.set_xlabel('percent of population')
    ax1.set_ylabel('precision', color='b')
    ax1.set_ylim(0,1.05)
    ax2 = ax1.twinx()
    ax2.plot(pct_above_per_thresh, recall_curve, 'r')
    ax2.set_ylabel('recall', color='r')
    ax2.set_ylim(0,1.05)
    
    name = model_name
    plt.title(name)
    plt.show()
    plt.clf()

In [ ]:
def precision_at_k(y_true, y_scores,k):
    
    threshold = np.sort(y_scores)[::-1][int(k*len(y_scores))]
    y_pred = np.asarray([1 if i >= threshold else 0 for i in y_scores ])
    return precision_score(y_true, y_pred)

In [ ]:
plot_precision_recall_n(expected,y_scores, 'LR')

In [ ]:
p_at_1 = precision_at_k(expected,y_scores, 0.01)
print('Precision at 1%: {:.2f}'.format(p_at_1))

# Assess Model Against Baselines

- Back to [Table of Contents](#Table-of-Contents)

It is important to check our model against a reasonable **baseline** to know how well our model is doing. Without any context, 78% accuracy can sound really great... but it's not so great when you remember that you could do almost that well by declaring everyone will not need benefits in the next year, which would be stupid (not to mention useless) model. 

A good place to start is checking against a *random* baseline, assigning every example a label (positive or negative) completely at random. 

In [ ]:
random_score = [random.uniform(0,1) for i in enumerate(y_test)] 
random_predicted = np.array( [calc_threshold(score,0.5) for score in random_score] )
random_p_at_5 = precision_at_k(expected,random_predicted, 0.01)
print('Precision at 1% (random): {:.2f}'.format(random_p_at_5))

## Decision Tree

Linear regression and logistic regression models fail in situations where the relationship between features and outcome is nonlinear or where features interact with each other. Tree based models split the data multiple times according to certain cutoff values in the features. Through splitting, different subsets of the dataset are created, with each instance belonging to one subset. The final subsets are called terminal or leaf nodes and the intermediate subsets are called internal nodes or split nodes. To predict the outcome in each leaf node, the average outcome of the training data in this node is used. Trees can be used for classification and regression. Here is more info on how to interpret the tree: http://engineering.pivotal.io/post/interpreting-decision-trees-and-random-forests/

In [ ]:
clf_gini = DecisionTreeClassifier(criterion = "gini", random_state = 100,
                               max_depth=3, min_samples_leaf=5)
clf_gini.fit(X_train, y_train)

In [ ]:
#import graphviz 
dot_data = tree.export_graphviz(clf_gini, out_file=None) 
graph = graphviz.Source(dot_data) 

Unfortunately this package is not installed in our environment yet. You can create your own virtual environment and install it there typing conda install python-graphviz in the terminal. It look should something like this: https://scikit-learn.org/stable/modules/tree.html

# Supervised Learning: Document Classification

Previously, we used topic modeling to infer relationships of abstracts of funded research projects. That is an example of unsupervised learning: we were looking to uncover structure in the form of topics, or groups of agencies, but we did not necessarily know the ground truth of how many groups we should find or which agencies belonged in which group.  

Now we turn our attention to supervised learning. In supervised learning, we have a *known* outcome or label (*Y*) that we want to produce given some data (*X*), and in general, we want to be able to produce this *Y* when we *don't* know it, or when we *only* have *X*. 

In order to produce labels we need to first have examples our algorithm can learn from, a "training set." In the context of text analysis, developing a training set can be very expensive, as it can require a large amount of human labor or linguistic expertise. **Document classification** is an example of supervised learning in which want to characterize our documents based on their contents (*X*). A common example of document classification is spam e-mail detection. Another example of supervised learning in text analysis is *sentiment analysis*, where *X* is our documents and *Y* is the state of the author. This "state" is dependent on the question you're trying to answer, and can range from the author being happy or unhappy with a product to the author being politically conservative or liberal. Another example is *part-of-speech tagging* where *X* are individual words and *Y* is the part-of-speech. 

In this section, we'll train a classifier to classify abstract of grant proposals. Let's see if we can label a new abstract as belonging to a high paying research grant or low paying grant"

## Supervised Learning - Prepare the Data

In [ ]:
# Load data into dataframe
abstr = (pd.read_csv(str(data_dir) + "abstracts/FedRePORTER_PRJABS_C_FY2017.csv",
                  skipinitialspace=True, encoding='utf-8'))

In [ ]:
# Merge with project data
proj = (pd.read_csv(str(data_dir) + "projects/FedRePORTER_PRJ_C_FY2017.csv",
                  skipinitialspace=True, encoding='utf-8'))
abstracts = pd.merge(abstr, proj[['FY_TOTAL_COST', 'PROJECT_ID']], on='PROJECT_ID', how='inner')

In [ ]:
# make a mask column we can use to flag rows with facility type in our types of interest.
abstracts['value'] = np.where((abstracts['FY_TOTAL_COST'] >= 100000) , 'High', 'Low')

In [ ]:
# drop missing values
abstracts = abstracts.dropna()

In [ ]:
# Check data
abstracts.head()

In [ ]:
# split into train and test sets.
df_train, df_test = train_test_split(abstracts, test_size=0.20, random_state=17)

In [ ]:
# look at our training set.
df_train.head()

In [ ]:
# make sure we only have the types we expect.
df_train['value'].unique()

In [ ]:
# look at the counts for each value.
Counter(df_train['value'].values)

In [ ]:
# look at our testing set.
df_test.head()

In [ ]:
# make sure we only have the facility types we expect.
df_test['value'].unique()

In [ ]:
# look at the counts for each value.
Counter(df_test['value'].values)

## Prepare Data for Document Classification

In order to feed out data into a classifier, we need to pull out the labels (*Y*) and a clean corpus of documents (*X*) for our training and testing sets. You want to clean the text data before doing this.

In [ ]:
# prepare training data - get labels we'll train on.
train_labels = df_train.value.values

# prepare training data - clean text.
train_corpus = np.array(df_train.ABSTRACT.values)

# prepare testing data - get labels we'll train on.
test_labels = df_test.value.values

# prepare testing data - clean text.
test_corpus = np.array(df_test.ABSTRACT.values)

# make list of all labels across train and test (should just be 'income' and 'health')
labels = np.append(train_labels, test_labels)

Just as we had done in the unsupervised learning context, we have to transform our data. This time we have to transform our testing and training set into two different bags of words. The classifier will learn from the training set, and we will evaluate the classifier's performance on the testing set.

First, we create a CountVectorizer that we'll use to convert our text documents to matrices of features based on words contained within our corpus.

In [ ]:
#parameters for vectorizer 
ANALYZER = "word" #unit of features are single words rather then phrases of words 
STRIP_ACCENTS = 'unicode'
TOKENIZER = None
NGRAM_RANGE = (0,2) #Range for pharases of words
MIN_DF = 0.01 # Exclude words that have a frequency less than the threshold
MAX_DF = 0.8  # Exclude words that have a frequency greater then the threshold 

vectorizer = CountVectorizer( analyzer = ANALYZER,
                              tokenizer = None, # alternatively tokenize_and_stem but it will be slower 
                              ngram_range = NGRAM_RANGE,
                              stop_words = stopwords.words( 'english' ),
                              strip_accents = STRIP_ACCENTS,
                              min_df = MIN_DF,
                              max_df = MAX_DF )

Next, we create a TF-IDF transformer, and create our bags of words, then weight them using TF-IDF.

In [ ]:
NORM = None          # turn on normalization flag
SMOOTH_IDF = True    # prevents division by zero errors
SUBLINEAR_IDF = True # replace TF with 1 + log(TF)
USE_IDF = True       # flag to control whether to use TFIDF

transformer = TfidfTransformer( norm = NORM,
                                smooth_idf = SMOOTH_IDF,
                                sublinear_tf = True )

# timing code - start!
start_time = time.time()

# get the bag-of-words for train and test from the vectorizer and
# then use TFIDF to limit the tokens found throughout the text 
train_bag_of_words = vectorizer.fit_transform( train_corpus ) #using all the data on for generating features!! Bad!
test_bag_of_words = vectorizer.transform( test_corpus )

# if we use IDF, compute it here.
if USE_IDF:
    train_tfidf = transformer.fit_transform(train_bag_of_words)
    test_tfidf = transformer.transform(test_bag_of_words)

# Get list of the feature names, for passing to our model.
features = vectorizer.get_feature_names()

# timing code - done!
print('Time Elapsed: {0:.2f}s'.format( time.time() - start_time ) )

We cannot pass the labels directly to the classifier. Instead, we to encode them as 0s and 1s using the `labelencoder` part of `sklearn`. 

In [ ]:
# relabel our labels as a 0 or 1
le = preprocessing.LabelEncoder() 
le.fit(labels)
labels_binary = le.transform(labels)

In [ ]:
list(zip(labels,labels_binary))

We also need to create arrays of indices so we can access the training and testing sets accordingly.

In [ ]:
train_size = df_train.shape[ 0 ]
train_set_idx = np.arange( 0, train_size )
test_set_idx = np.arange( train_size, len( labels ) )
train_labels_binary = labels_binary[ train_set_idx ]
test_labels_binary = labels_binary[ test_set_idx ]

## Model Training - Train Document Classification Model

The classifier we are using in the example is LogisticRegression. As we saw in the Machine Learning tutorial, first we decide on a classifier, then we fit the classifier to the data to create a model. We can then test our model on the test set by passing the features (*X*) from our test set to get predicted labels. The model will output the probability of each abstract being classified as low or high. 

In [ ]:
# create our LogisticRegression classifier.
clf = LogisticRegression(penalty='l1')

# train the classifer to create our model.
mdl = clf.fit( train_tfidf, labels_binary[ train_set_idx ] )

# create scores for each of the documents predicting whether each refers to 
#     an income or health agency
y_score = mdl.predict_proba( test_tfidf )

## Model Evaluation - Precision and Recall

Now that we have calculated a score for each of our facility types of interest, we look at how well our model performed by outputting precision and recall curves at different cutoffs.

In [ ]:
plot_precision_recall_n( labels_binary[ test_set_idx ], y_score[:,1], 'LR' )

Alternatively, we can try to maximize the entire precision-recall space. In this case we need a different metric - "Area Under Curve" (AUC). The AUC shows how accurate our scores are under different cutoff thresholds. The model will output a score between 0 and 1. We specify a  range of cutoff values and label all of the examples as 0 or 1 based on whether they are above or below each cutoff value. The closer our scores are to the true values, the more resilient they are to different cutoffs. For instance, if our scores were perfect, our AUC would be 1. 

In [ ]:
def plot_precision_recall(y_true,y_score):
    """
    Plot a precision recall curve
    
    Parameters
    ----------
    y_true: ls
        ground truth labels
    y_score: ls
        score output from model
    """
    precision_curve, recall_curve, pr_thresholds = precision_recall_curve(y_true,y_score[:,1])
    plt.plot(recall_curve, precision_curve)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    auc_val = auc(recall_curve,precision_curve)
    print('AUC-PR: {0:1f}'.format(auc_val))
    plt.show()
    plt.clf()

In [ ]:
plot_precision_recall(labels_binary[test_set_idx],y_score)

## Model Evaluation - Feature Importances

Next, we look at the importance of different features (words) in our model.

The function that will calculate these:

In [ ]:
def display_feature_importances( coef, features, labels, num_features = 10 ):

    """
    output feature importances
    
    Parameters
    ----------
    coef: numpy
        feature importances
    features: ls 
        feature names
    labels: ls
        labels for the classifier
    num_features: int
        number of features to output (default 10)
    
    Example
    --------
    
    
    """
    coef = mdl.coef_.ravel()

    dict_feature_importances = dict( zip(features, coef) )
    orddict_feature_importances = OrderedDict( 
                                    sorted(dict_feature_importances.items(), key=lambda x: x[1]) )

    ls_sorted_features  = list(orddict_feature_importances.keys())

    label0_features = ls_sorted_features[:num_features] 
    label1_features = ls_sorted_features[-num_features:] 

    print(labels[0],label0_features)
    print(labels[1], label1_features)

In [ ]:
display_feature_importances(mdl.coef_.ravel(), features, ['high','low'])

The feature importances give us the words which are the most relevant for distinguishing the type of grant (between high or low funded grant). Some of these make sense, but some don't make as much sense, or seem to be artifacts that we should remove. 

## Machine Learning Pipeline
*[Go back to Table of Contents](#Table-of-Contents)*

When working on machine learning projects, it is a good idea to structure your code as a modular **pipeline**, which contains all of the steps of your analysis, from the original data source to the results that you report, along with documentation. This has many advantages:
- **Reproducibility**. It's important that your work be reproducible. This means that someone else should be able
to see what you did, follow the exact same process, and come up with the exact same results. It also means that
someone else can follow the steps you took and see what decisions you made, whether that person is a collaborator, 
a reviewer for a journal, or the agency you are working with. 
- **Ease of model evaluation and comparison**.
- **Ability to make changes.** If you receive new data and want to go through the process again, or if there are 
updates to the data you used, you can easily substitute new data and reproduce the process without starting from scratch.

## Resources
*[Go back to Table of Contents](#Table-of-Contents)*

- Hastie et al.'s [The Elements of Statistical Learning](http://statweb.stanford.edu/~tibs/ElemStatLearn/) is a classic and is available online for free.
- James et al.'s [An Introduction to Statistical Learning](http://www-bcf.usc.edu/~gareth/ISL/), also available online, includes less mathematics and is more approachable.
- Wu et al.'s [Top 10 Algorithms in Data Mining](http://www.cs.uvm.edu/~icdm/algorithms/10Algorithms-08.pdf).